Cross-Session on Multiple Datasets

In [ ]:
import warnings

import matplotlib.pyplot as plt
import seaborn as sns
from mne.decoding import CSP
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.pipeline import make_pipeline

import moabb
from moabb.datasets import BNCI2014_001, BNCI2015_001
from loaddata.pan2023 import Pan2023
from moabb.evaluations import CrossSessionEvaluation
from moabb.paradigms import LeftRightImagery


warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=RuntimeWarning)
moabb.set_log_level("info")

Loading Dataset

In [ ]:
path = r'E:\工作进展\小论文2023会议\数据处理python\datasets'
datasets = [BNCI2014_001(path=path), BNCI2015_001(path=path), Pan2023(path=path)]
for d in datasets:
    print(d.subject_list)

Choose Paradigm

In [ ]:
paradigm = LeftRightImagery(resample=200, fmin=8, fmax=30, tmin=0, tmax=4)

Create Pipelines

In [ ]:
from pre_processing.preprocessing import Pre_Processing
from transfer_learning.tl_classifier import TL_Classifier

pipelines = {}

pipelines["CSP+LDA"] = make_pipeline(CSP(n_components=8), LDA())

pipelines["RG+LR"] = make_pipeline(
    Covariances(), TangentSpace(), LogisticRegression(solver="lbfgs")
)

Evaluation

In [ ]:
overwrite = True  # set to True if we want to overwrite cached results

evaluation = CrossSessionEvaluation(
    paradigm=paradigm, datasets=datasets, suffix="examples", overwrite=overwrite
)
results = evaluation.process(pipelines)

print(results.head())